# คำนวณค่าตัวแปรที่ใช้ในการโมเสค ช่วยให้ประหยัดเวลาในการคำนวณ
* เซฟตัวแปรเก็บไว้ แล้วนำมาใช้ต่อในไฟล์โค้ดหลัก

In [1]:
import numpy as np
import numpy.ma as ma
import cartopy.crs as ccrs
import cartopy
import matplotlib.pyplot as pl
import wradlib as wrl
from wradlib.io import read_generic_netcdf
import pyart
import sys
import warnings
warnings.filterwarnings('ignore')
import glob
import os
import re
import time
import gzip
import shutil


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [2]:
print(wrl.__version__)
print(sys.version)
print(pyart.__version__)

1.14.0
3.8.13 | packaged by conda-forge | (default, Mar 25 2022, 05:59:45) [MSC v.1929 64 bit (AMD64)]
0.0.0


In [3]:
# เพื่อแตกไฟล์ gzip ให้เป็น uf 
def decompress_gzip(input_file, output_file):
    try:
        with gzip.open(input_file, 'rb') as f_in:
            with open(output_file, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        print(f"File decompressed successfully: {output_file}")
        return True
    except Exception as e:
        print(f"An error occurred while decompressing {input_file}: {str(e)}")
        return False

def check_and_decompress(path, mos_time):
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith('.bz2') and mos_time in file:
                bz2_file = os.path.join(root, file)
                uf_file = os.path.splitext(bz2_file)[0]
                if not os.path.exists(uf_file):
                    print(f"Decompressing {bz2_file}")
                    decompress_gzip(bz2_file, uf_file)        

In [4]:
# ลบไฟล์นามสกุล .uf กับ .nc ที่แตกไฟล์มาและที่ได้จากประมวลผล
def cleanup_files(path, extensions_to_delete=['.uf', '.nc']):
    for root, dirs, files in os.walk(path):
        for ext in extensions_to_delete:
            for file in glob.glob(os.path.join(root, f'*{ext}')):
                try:
                    os.remove(file)
                    print(f"Deleted: {file}")
                except Exception as e:
                    print(f"Error deleting {file}: {str(e)}")

In [5]:
def remove_noise_flare_SNR(file_in, file_out):
    #radar = pyart.io.read_uf(file_in)
    radar =pyart.io.auto_read.read(file_in)
    
    snr = pyart.retrieve.calculate_snr_from_reflectivity(radar, refl_field='reflectivity',toa=2500.0) #ค่าตั้งต้นคือ 15000 ทดลองเอง 
#ลอง 12500 ไม่เวิร์ค ลอง 5000 ก็ไม่เวิร์คเรดาร์ชุมพรฝนหาย ลอง 1000 เวิร์ค ฝนมี แต่เหมือนบางลง ตกลงใช้ toa 2500 เพราะฝนหายน้อยกว่า ได้ฝนครบ

    radar.add_field('signal_to_noise_ratio', snr, replace_existing=True)

    # กรองสัญญาณรบกวน ด้วยการใช้ SNR ใช้ gatefilter
    # ตรงนี้ต้องใช้ if กรณีที่ไม่มีค่า dual-pol ฟิวด์ ให้กลับไปใช้ SNR ที่คำนวณได้ ในการทำ texture
    gtfilter = pyart.filters.moment_and_texture_based_gate_filter(radar, phi_field='differential_phase')
    gtfilter.exclude_below('signal_to_noise_ratio', 2) #ใช้ค่า snr  = 10 
    gtfilter.exclude_above('signal_to_noise_ratio', 70) #ใช้ค่า snr  = 60 

    # แอดฟิวด์ที่ได้กรองสัญญาณรบกวนออกไป เอาโค้ดมาจาก  https://github.com/ARM-DOE/pyart/issues/763
    nf = radar.fields['corrected_reflectivity']
    nf['data'] = np.ma.masked_where(gtfilter.gate_excluded , nf['data'])
    radar.add_field('filtered_refectivity', nf, replace_existing=True)

    # ส่งออก radar object เพื่อนำฟิวด์ที่กรองสัญญาณรบกวนออกไปnetcdf เพื่อใช้ใน wradlib
    #file = './0data/PHI201807171100SNR'
    pyart.io.write_cfradial(file_out, radar, format='NETCDF4')
    
    #return 

In [6]:
#ฟังค์ชั่นรับค่าพารามิเตอร์เพื่อนำไปคำนวณพิกัด utm
def get_geom(vol, i=0) :
    # เก็บตัวแปร variables
    v = vol['variables']

    # เก็บพิกัด
    lon = v['longitude']['data']
    lat = v['latitude']['data']
    alt = v['altitude']['data']
    radar_location = (lon, lat)    
    
    # อินเด็กซ์ของการกวาดในแต่ละมุมยก swstart=ค่าอินเด็กซ์แรกของแต่ละมุมยก swend= ค่าอินเด็กซ์สุดท้ายของแต่ละมุมยก
    swstart = v['sweep_start_ray_index']['data']
    swend = v['sweep_end_ray_index']['data']

     # ค่าเริ่มและหยุดในการกวาดมุมยกแรก i=0
    i = 0
    i1 = swstart[i]
    i2 = swend[i]

    # มุมอซิมัท
    azi = v['azimuth']['data']
    az = azi[i1:i2]

    # ระยะห่างระหว่าง gate/bin
    r = v['range']['data']
    rg =r[i1:i2]

    # ค่ามุมยก
    el = v['fixed_angle']['data']
    elev=el[i] #i = 0 มุมแรก
    
    return az, rg, elev, radar_location

In [7]:
# คำนวณค่าพิกัดเรดาร์ด้วยการแปลงจากโพลาร์ไปเป็นกริด
proj_utm = wrl.georef.epsg_to_osr(32647)
def cal_coord(rg, az, elev, radar_location, proj_utm):
    coord = wrl.georef.spherical_to_centroids(rg, az, elev, radar_location, proj=proj_utm)
    coord = coord[..., 0:2]
    coord = coord.reshape(-1, coord.shape[-1])
    
    return coord

In [8]:
#คำนวณค่าพิกัดขอบเขตการโมเสค ตามพิกัดเรดาร์
def bbox(*args):
    """Get bounding box from a set of radar bin coordinates"""
    x = np.array([])
    y = np.array([])
    for arg in args:
        x = np.append(x, arg[:, 0])
        y = np.append(y, arg[:, 1])
        #print( ': ', x.min(), x.max(), ': ', y.min(), y.max())
    xmin = x.min()
    xmax = x.max()
    ymin = y.min()
    ymax = y.max()

    return xmin, xmax, ymin, ymax

In [9]:
# คำนวณค่ากริดผลลัพธ์ จากขอบเขตที่ได้จาก bbox ฟังก์ชัน ของทุกเรดาร์
def cal_grid_out(coord_all, num_xy):
    # define target grid for composition
    #xmin, xmax, ymin, ymax = bbox(coord_all)    
    # ฟังก์ชั่นนี้จะใช้วิธีการ fix ค่าพิกัด extent ของพื้นที่การโมเสคทั้งประเทศ เพราะต้องการจะทำให้ทุกช่วงเวลาของการกวาด มีขอบเขตที่ตรงกัน
    # ค่า xmin, xmax, ymin, ymax ได้มาจากการคำนวณหาค่า ด้วยการใช้เรดาร์ เชียงราย ลำพูน พิษณุโลก ชัยนาท เพื่อหาขอบเขต เหนือ ใต้ ออก ตก
    xmin, xmax, ymin, ymax= 150000.00, 925000.00, 1420000.00, 2500000.00
    print('>>>>>xmin, xmax, ymin, ymax of cal_grid_out: ',  xmin, xmax, ymin, ymax)
    x = np.linspace(xmin, xmax + 1000.0, num_xy)
    y = np.linspace(ymin, ymax + 1000.0, num_xy)
    grid_coords = wrl.util.gridaspoints(y, x)
    
    return x, y, grid_coords

In [10]:
def check_files_mos(path, mos_time): 
    print('\n\n--->โมเสคทั้งหมด แม้ว่าจะมีไม่ครบ 4 ครั้ง/ชั่วโมง')
    
    # ตรวจสอบไฟล์ gzip เพื่อแตกไฟล์เป็น uf
    check_and_decompress(path, mos_time)

    # ค้นหาสถานีที่มีการสแกน แม้ว่าจะมีไม่ครบ 4 ครั้ง/ชั่วโมง
    fd_name = os.listdir(path)
    numFiles=[]
    fd_run=[]
    for i, fd in enumerate(fd_name):
        j=0
        for file in os.listdir(path+'/'+ fd):
            if file.endswith('.uf'):
                j+=1            
        if j!=0 : 
            numFiles.append(j)
            fd_run.append(fd)

    #-ลิสต์ไฟล์ตามเวลาที่สแกน เพื่อนำไปโมเสคราย 15 นาที

    min_fn = ["00", "15", "30", "45"] #สแกนนาที ที่ [00, 15, 30, 45]
    files_total=[]

    for min_s in min_fn:
        files_total.append('scan minute: '+ min_s)
        j=0
        files=[]
        fd_n=[]
        for i, fd in enumerate(fd_run):
            #print(fd+'---')        
            for file in os.listdir(path+'/'+ fd):
                if file.endswith('.uf') and re.search(r"2018(\d{6})"+min_s, file) :
                #if file.endswith('.bz2') and re.search(r"2018(\d{6})"+min_s, file) :
                    #print(i, fd, file)
                    files.append(file) #เพิ่มเฉพาะไฟล์ที่เจอในลิสต์ 
                    fd_n.append(fd) #เพิ่มเฉพาะโฟลเดอร์ที่เจอ *uf ในลิสต์
                    j+=1
                    #print(file)     
        files_total.append(j)
        files_total.append(fd_n)
        files_total.append(files)
    
    return fd_run, files_total

#--นำไปโมเสคได้ ราย 15 นาที แล้วนำไปทำฝนสะสมรายชั่วโมง กรณีที่ต้องการการสแกนที่ครบถ้วน

In [11]:
# ตั้งค่าผลิตภัณฑ์ฝนตรงนี้
path = '../1data/0radar/0Mosaic15min_4rads_1hour_Sentihn0_good/'
#path = '../1data/0radar/0Mosaic15min_4rads_1hour_Sentihn1/'
#path = '../1data/0radar/0Mosaic15min_4rads_1hour_Bebinca/'
#path = '../1data/0radar/0Mosaic15min_4rads_1hour/'
#path = '../1data/0radar/0Mosaic15min_4rads_list_files/'

#เปลี่ยนขนาดกริดตรงนี้
gridsize_new=2000 #ขนาดกริดใหม่ หน่วยเมตร

gridsize_org = 1000 #ขนาดกริดตั้งต้น
num_grid_org = gridsize_org**2 #จำนวนกริดตั้งต้น
num_xy = int(num_grid_org/gridsize_new) #จำนวนกริดในแนวแกน x และy
num_grid_new = int(num_grid_org/gridsize_new)**2 #จำนวนกริดใหม่ทั้งหมด

# ตรวจสอบสถานีและจำนวนไฟล์ที่จะใช้ในการโมเสคแต่ละช่วงเวลา
#fd_run, files_total = check_files_mos(path)
#print(fd_run)
#print(files_total)

In [12]:
#เลือกเวลาที่ต้องการโมเสค ปี เดือน วัน ชั่วโมง นาที
mos_time='201807171130'
scan_time=mos_time[-2:] 

# ตรวจสอบสถานีและจำนวนไฟล์ที่จะใช้ในการโมเสคแต่ละช่วงเวลา
fd_run, files_total = check_files_mos(path,mos_time)
print(fd_run)

# Iterate through files_total to find the index of the matching scan minute
index = None
for i, item in enumerate(files_total):
    if isinstance(item, str) and item == f'scan minute: {scan_time}':
        index = i
        break

# Check if a match was found and print the index
if index is not None:
    print('สแกนนาที: ', files_total[index])
    print('จำนวนสถานีที่มีข้อมูล: ', files_total[index+1])
    print('ชื่อสถานีที่มีข้อมูล: ', files_total[index+2])
    num_sta_available=files_total[index+1]
    name_sta_available=files_total[index+2]
    list_name_uf_sta_available=files_total[index+3]

else:
    print(f"'scan minute: {scan_time}' not found in the list")



--->โมเสคทั้งหมด แม้ว่าจะมีไม่ครบ 4 ครั้ง/ชั่วโมง
Decompressing ../1data/0radar/0Mosaic15min_4rads_1hour_Sentihn0_good/CHN\CHN240@201807171130.uf.bz2
File decompressed successfully: ../1data/0radar/0Mosaic15min_4rads_1hour_Sentihn0_good/CHN\CHN240@201807171130.uf
Decompressing ../1data/0radar/0Mosaic15min_4rads_1hour_Sentihn0_good/CRI\CRI240@201807171130.uf.bz2
File decompressed successfully: ../1data/0radar/0Mosaic15min_4rads_1hour_Sentihn0_good/CRI\CRI240@201807171130.uf
Decompressing ../1data/0radar/0Mosaic15min_4rads_1hour_Sentihn0_good/LMP\LMP240@201807171130.uf.bz2
File decompressed successfully: ../1data/0radar/0Mosaic15min_4rads_1hour_Sentihn0_good/LMP\LMP240@201807171130.uf
Decompressing ../1data/0radar/0Mosaic15min_4rads_1hour_Sentihn0_good/PHS\PHS240@201807171130.uf.bz2
File decompressed successfully: ../1data/0radar/0Mosaic15min_4rads_1hour_Sentihn0_good/PHS\PHS240@201807171130.uf
['CHN', 'CRI', 'LMP', 'PHS']
สแกนนาที:  scan minute: 30
จำนวนสถานีที่มีข้อมูล:  4
ชื่อสถานีท

In [13]:
sta=name_sta_available
fn_in=list_name_uf_sta_available
#fn_in=list_name_bz2_sta_available
fn_out = [item + '_SNR.nc' for item in sta]

In [14]:
fn_in

['CHN240@201807171130.uf',
 'CRI240@201807171130.uf',
 'LMP240@201807171130.uf',
 'PHS240@201807171130.uf']

In [15]:
fn_out

['CHN_SNR.nc', 'CRI_SNR.nc', 'LMP_SNR.nc', 'PHS_SNR.nc']

In [16]:
print('>>1. กรอง noise + flare ใน pyart...')
for i, sta_n in enumerate(sta):    
    file_in= path+sta_n+'/'+fn_in[i]
    file_out= path+sta_n+'/'+fn_out[i]
    print('--> noise+flare removing : ', file_in)
    
    # ฟังก์ชันกรอง noise+flare ใน pyart แล้วเซฟเป็นไฟล์ใหม่เพื่อนำไปใช้ใน wradlib
    try:
        remove_noise_flare_SNR(file_in, file_out)
    except:
        print('fail: ', file_in)
        pass

>>1. กรอง noise + flare ใน pyart...
--> noise+flare removing :  ../1data/0radar/0Mosaic15min_4rads_1hour_Sentihn0_good/CHN/CHN240@201807171130.uf
--> noise+flare removing :  ../1data/0radar/0Mosaic15min_4rads_1hour_Sentihn0_good/CRI/CRI240@201807171130.uf
--> noise+flare removing :  ../1data/0radar/0Mosaic15min_4rads_1hour_Sentihn0_good/LMP/LMP240@201807171130.uf
--> noise+flare removing :  ../1data/0radar/0Mosaic15min_4rads_1hour_Sentihn0_good/PHS/PHS240@201807171130.uf


In [17]:
##################################################################

#คำนวณค่า extent ของเรดาร์ทุกสถานี เพื่อสร้างกริด utm ผลลัพธ์
print('\n\n2.************************************************\n')
print('คำนวณค่า extent ของข้อมูลเรดาร์ทุกสถานี\n')
coord_list = []
for i, sta_n in enumerate(sta):

    print(path+sta_n+'/'+fn_out[i])
    file= path+sta_n+'/'+fn_out[i]
    
    #อ่าน vol เข้ามา
    vol=read_generic_netcdf(file)
    
    #รับค่าพารามิเตอร์เพื่อคำนวณพิกัดจากฟังก์ชั่น
    az, rg, elev, radar_location = get_geom(vol, 0) #el=0 เท่ากับมุมยกที่ 1 
    
    #แปลงค่าพิกัดโพลาร์ เป็น กริด utm เพื่อให้ได้  coord แต่ละสถานี ที่มีสองหลัก  
    proj_utm = wrl.georef.epsg_to_osr(32647)
    
    try:    
        coord = cal_coord(rg, az, elev, radar_location, proj_utm)        
    except Exception as e:
        print(f"Error processing radar data for {sta_n}: {e}")
        continue
    
    # Append the coordinates of the current radar to the list
    coord_list.append(coord)
    
# Concatenate coordinates from all radars
coord_all = np.concatenate(coord_list, axis=0)

# Calculate the overall grid extents
x, y, grid_coords = cal_grid_out(coord_all, num_xy)


#data_qi_gridded = np.empty((len(sta), 1000000))  #1000000 เป็นค่า  quality_gridded.shape
#data_gridded = np.empty((len(sta), 1000000))  #1000000 เป็นค่า  quality_gridded.shape   

data_qi_gridded = np.empty((len(sta), num_grid_new))  #1000000 เป็นค่า  quality_gridded.shape
data_gridded = np.empty((len(sta), num_grid_new))  #1000000 เป็นค่า  quality_gridded.shape  



2.************************************************

คำนวณค่า extent ของข้อมูลเรดาร์ทุกสถานี

../1data/0radar/0Mosaic15min_4rads_1hour_Sentihn0_good/CHN/CHN_SNR.nc
../1data/0radar/0Mosaic15min_4rads_1hour_Sentihn0_good/CRI/CRI_SNR.nc
../1data/0radar/0Mosaic15min_4rads_1hour_Sentihn0_good/LMP/LMP_SNR.nc
../1data/0radar/0Mosaic15min_4rads_1hour_Sentihn0_good/PHS/PHS_SNR.nc
>>>>>xmin, xmax, ymin, ymax of cal_grid_out:  150000.0 925000.0 1420000.0 2500000.0


# เซฟค่าตัวแปรที่ใช้ในการโมเสคโปรแกรมหลัก

In [18]:
# Folder name
folder_name = "0variables_mosaic"

# Create folder if it doesn't exist
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Save variables to files
np.save(os.path.join(folder_name, 'coord_list.npy'), coord_list)
np.save(os.path.join(folder_name, 'coord_all.npy'), coord_all)
np.save(os.path.join(folder_name, 'x.npy'), x)
np.save(os.path.join(folder_name, 'y.npy'), y)
np.save(os.path.join(folder_name, 'grid_coords.npy'), grid_coords)
np.save(os.path.join(folder_name, 'num_xy.npy'), np.array([num_xy]))

print(f"Variables saved in folder: {folder_name}")

Variables saved in folder: 0variables_mosaic


In [19]:
'''
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# Create a custom colormap
def create_custom_cmap(base_cmap='viridis'):
    cmap = plt.get_cmap(base_cmap)
    colors = cmap(np.arange(cmap.N))
    colors[0, :] = [0.9, 0.9, 0.9, 1.0]  # Set the color of the first color (value 0) to light gray
    custom_cmap = ListedColormap(colors)
    return custom_cmap

# Plotting function for each station
def plot_station_data(data_gridded, x, y, station_name, custom_cmap):
    # Mask the data values that are NaN
    masked_data = np.ma.masked_where(np.isnan(data_gridded), data_gridded)

    # Plotting the masked data
    plt.figure(figsize=(10, 8))
    ax = plt.subplot(111, aspect="equal")
    plt.pcolormesh(x, y, masked_data.reshape((len(y), len(x))), cmap=custom_cmap, vmin=0, vmax=5)
    plt.colorbar(label="Rainfall Intensity (mm)")
    
    # Plot radar boundaries
    #circle = plt.Circle(radar_location, radius, color='gray', alpha=0.3, fill=False)
    #ax.add_patch(circle)

    plt.grid()
    plt.xlim(min(x), max(x))
    plt.ylim(min(y), max(y))
    plt.title(f"Radar Rainfall Depth - {station_name}")
    plt.xlabel("UTM Easting (m)")
    plt.ylabel("UTM Northing (m)")
    plt.show()


# Create the custom colormap
custom_cmap = create_custom_cmap()

# Plot data for each station
for i, sta_n in enumerate(sta):
    plot_station_data(data_gridded[i], x, y, sta_n, custom_cmap)
'''

'\nimport numpy as np\nimport matplotlib.pyplot as plt\nfrom matplotlib.colors import ListedColormap\n\n# Create a custom colormap\ndef create_custom_cmap(base_cmap=\'viridis\'):\n    cmap = plt.get_cmap(base_cmap)\n    colors = cmap(np.arange(cmap.N))\n    colors[0, :] = [0.9, 0.9, 0.9, 1.0]  # Set the color of the first color (value 0) to light gray\n    custom_cmap = ListedColormap(colors)\n    return custom_cmap\n\n# Plotting function for each station\ndef plot_station_data(data_gridded, x, y, station_name, custom_cmap):\n    # Mask the data values that are NaN\n    masked_data = np.ma.masked_where(np.isnan(data_gridded), data_gridded)\n\n    # Plotting the masked data\n    plt.figure(figsize=(10, 8))\n    ax = plt.subplot(111, aspect="equal")\n    plt.pcolormesh(x, y, masked_data.reshape((len(y), len(x))), cmap=custom_cmap, vmin=0, vmax=5)\n    plt.colorbar(label="Rainfall Intensity (mm)")\n    \n    # Plot radar boundaries\n    #circle = plt.Circle(radar_location, radius, color=\